In [96]:
import spacy
import pandas as pd
import numpy as np



In [57]:
text_data = pd.read_csv('clinical_trials_dataset.csv')
#text_data = text_data.dropna(axis=0,how='any').reset_index(drop=True)
text_data = text_data[['OfficialTitle','StudyType','StartDate',
           'Condition','ConditionAncestorTerm',
           'ArmGroupInterventionName','InterventionArmGroupLabel','InterventionDescription']].copy()
#text_data[text_data['ArmGroupInterventionName'].str.contains('Genetic')]
text_data.isna().sum()

OfficialTitle                  53
StudyType                       0
StartDate                      42
Condition                       0
ConditionAncestorTerm         571
ArmGroupInterventionName     1128
InterventionArmGroupLabel    1124
InterventionDescription      1158
dtype: int64

In [110]:
#deal with nan values column by column
test = text_data.copy()
test = test.fillna({'StartDate':'unknown'}).copy()
test = test.fillna({'OfficialTitle':'unknown'}).copy()
test['ConditionAncestorTerm'] = test['ConditionAncestorTerm'].fillna(test['Condition'])
test = test.fillna({'ArmGroupInterventionName':'other or NA'}).copy()
test = test.fillna({'InterventionArmGroupLabel':'other or NA'}).copy()
test = test.fillna({'InterventionDescription':'other or NA'}).copy()
text_data = test.copy()

del test

In [112]:
#replace nan values with other appropriate values
text_data.isna().sum()

OfficialTitle                0
StudyType                    0
StartDate                    0
Condition                    0
ConditionAncestorTerm        0
ArmGroupInterventionName     0
InterventionArmGroupLabel    0
InterventionDescription      0
dtype: int64

In [136]:
text_data[(text_data['ArmGroupInterventionName'].str.contains('Genetic:')) & (text_data['StudyType']=='Interventional')]

,OfficialTitle,StudyType,StartDate,Condition,ConditionAncestorTerm,ArmGroupInterventionName,InterventionArmGroupLabel,InterventionDescription
0,"Phase I/IIa, First-in-human, Open-label, Singl...",Interventional,September 2024,Drug Resistant Epilepsy,Brain Diseases|Central Nervous System Diseases...,Genetic: lentiviral gene therapy,lentiviral gene therapy treatment (Interventio...,lentiviral gene therapy to treat drug resistan...
5,"A Phase I Study to Assess the Safety, Tolerabi...",Interventional,"February 7, 2022",Alzheimer's Disease|Mild Cognitive Impairment,Dementia|Brain Diseases|Central Nervous System...,Genetic: AAV2-BDNF Gene Therapy|Biological: AA...,Gene transfer of AAV2-BDNF|Gene transfer of AA...,AAV2-BDNF is a genetically engineered adeno-as...
7,A Phase I/II Clinical Trial of Hematopoietic S...,Interventional,"April 9, 2010",Lysosomal Storage Disease|Metachromatic Leukod...,"Metabolism, Inborn Errors|Genetic Diseases, In...",Genetic: OTL-200 Gene Therapy,OTL-200 Gene Therapy,Autologous hematopoietic stem/progenitor cells...
8,Phase I/II Clinical Trial of Autologous Hemato...,Interventional,"July 23, 2021",Severe Combined Immunodeficiency Due to RAG1 D...,Immune System Diseases|Primary Immunodeficienc...,Genetic: Gene therapy,Gene therapy,Patients will be infused with autologous CD34+...
12,A Phase I/II Gene Therapy Trial for X-CGD With...,Interventional,July 2013,X-linked Chronic Granulomatous Disease,Pathologic Processes|Phagocyte Bactericidal Dy...,Genetic: ex-vivo gene-therapy,ex-vivo gene-therapy,"transplantation autologous CD34+ cells, transd..."
...,...,...,...,...,...,...,...,...
5804,"An Open-label, Dose-escalation Study to Evalua...",Interventional,"May 5, 2023",Neovascular Age-related Macular Degeneration,Retinal Degeneration|Retinal Diseases|Eye Dise...,Genetic: SKG0106|Genetic: SKG0106|Genetic: SKG...,Dose Level 1|Dose Level 2|Dose Level 3,SKG0106 is a recombinant adeno-associated viru...
5858,WEST-KOaST Study: WES (Whole Exome Sequencing)...,Interventional,"December 1, 2023",Testicular Cancer,Endocrine Gland Neoplasms|Neoplasms by Site|Ne...,Genetic: Analysis of biological samples of sal...,First-degree family members of patients with t...,The project aims to collect biological samples...
5864,Phase II Study of Capecitabine in Combination ...,Interventional,November 2006,Esophageal Cancer,Gastrointestinal Neoplasms|Digestive System Ne...,Drug: capecitabine|Drug: oxaliplatin|Genetic: ...,Chemo|Chemo|Chemo|Chemo|Chemo|Chemo|Chemo|Chem...,Oral|IV|Correlative Study|Correlative Study|Co...
5887,Neoadjuvant Intratumoral Injection of Dendriti...,Interventional,"May 1, 2006",Breast Cancer,Neoplasms by Site|Neoplasms|Breast Diseases|Sk...,Biological: therapeutic autologous dendritic c...,Vaccine|Vaccine|Vaccine|Vaccine|Vaccine|Vaccin...,injected into the primary breast mass or palpa...


In [ ]:
#this is for condition names
#the split('|') must be already applied to cond_list, function does not do that
def minimum_cond_name(cond_list):
    con_size=[]
    for elem in text_data['ConditionAncestorTerm'][0].split('|'):
        con_size.append(elem.count(' '))
    return cond_list[con_size.index(min(con_size))]

In [ ]:
#add column with modified conditions
text_data['mod_ConditionAncestorTerm'] = [minimum_cond_name(con_term.split('|')) for con_term in text_data['ConditionAncestorTerm']]

In [183]:
#text_data[text_data['ArmGroupInterventionName'].str.contains('Genetic:')]
text_data.loc[5]['ArmGroupInterventionName']

'Genetic: AAV2-BDNF Gene Therapy|Biological: AAV2-BDNF Gene Therapy'

In [203]:
modified_armgroup_interventionName=[]
for elem_arm in text_data['ArmGroupInterventionName']:
    if 'Genetic:' in elem_arm:
        if '|' in elem_arm:
            for elem_arm_gene in elem_arm.split('|'):
                if 'Genetic:' in elem_arm_gene:
                    modified_armgroup_interventionName.append(elem_arm_gene)
        elif '|' not in elem_arm:
            modified_armgroup_interventionName.append(elem_arm)
    else:
        modified_armgroup_interventionName.append('not genetic') 


In [ ]:
modified_armgroup_interventionName

In [201]:
for i in text_data['mod_ConditionAncestorTerm'].value_counts().index:
    print(i)
    

Neoplasms by Histologic Type
Neoplasms by Site
Neoplasms, Glandular and Epithelial
Respiratory Tract Neoplasms
Pathologic Processes
Disease
Genital Neoplasms, Male
Neoplasms
Neuroendocrine Tumors
Disease Attributes
Intestinal Neoplasms
Carcinoma
Blood Coagulation Disorders, Inherited
Anemia, Hemolytic, Congenital
Heart Diseases
Carcinoma, Bronchogenic
Liver Diseases
Digestive System Neoplasms
Muscular Disorders, Atrophic
Vascular Diseases
Immune System Diseases
Retinal Degeneration
Behavioral Symptoms
Endocrine Gland Neoplasms
Glucose Metabolism Disorders
Pathological Conditions, Anatomical
Astrocytoma
Sphingolipidoses
Parkinsonian Disorders
Gastrointestinal Neoplasms
Joint Diseases
Colorectal Neoplasms
Neoplastic Processes
Hematologic Diseases
Bronchial Diseases
Retinal Diseases
Neoplasms, Neuroepithelial
Urologic Neoplasms
Neoplasms, Connective and Soft Tissue
Headache Disorders, Primary
Pneumonia, Viral
Infections
Cancer
Blood-Borne Infections
Skin Diseases
Schizophrenia Spectrum an

In [173]:
minimum_cond_name(text_data['ConditionAncestorTerm'][3].split('|'))

'Immune System Diseases'